In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def get_reviews(movienm, filename="", pagenum=10) :

    print("크롤링 시작!")

    from selenium import webdriver
    from selenium.webdriver.common.by import By
    import time
    import pandas as pd

    # 요건 신경쓰지 말것.
    # from selenium.webdriver.chrome.options import Options
    # option = Options()
    # option.add_experimental_option('excludeSwitches', ['enable-logging'])
    # option.add_argument('--remote-debugging-port=9222')

    # 최종 결과물을 담을 변수 생성
    result = pd.DataFrame(columns=["score", "likes", "comment"])

    #  네이버영화 url 접속
    #  (크롬드라이버 경로는 본인 환경에 맞게 알아서 설정하시길!)
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get("https://movie.naver.com/")  

    #  영화명 검색
    search_box = driver.find_element(By.CSS_SELECTOR, "#ipt_tx_srch")
    search_box.clear()
    search_box.send_keys(movienm)
    driver.find_element(By.CSS_SELECTOR, "#jSearchArea > div > button").click()

    #  영화 링크 타고 들어가기
    movie_href = driver.find_element(By.CSS_SELECTOR, "#old_content > ul.search_list_1 > li:first-child")
    movie_href = movie_href.find_element(By.CSS_SELECTOR, "dl>dt>a").click()

    #  '평점' 탭 클릭
    driver.find_element(By.CSS_SELECTOR, "#movieEndTabMenu > li:nth-child(5) > a").click()
    time.sleep(0.5)

    #  '관람객 평점만 보기' 체크박스 클릭
    iframe = driver.find_element(By.CSS_SELECTOR, "#pointAfterListIframe")
    driver.switch_to.frame(iframe)
    driver.find_element(By.CSS_SELECTOR, "#actualYnLable").click()
    time.sleep(0.5)

    #  해당 페이지의 리뷰들 스크래핑 : 평점, 좋아요수, 감상평
    score = [] # 평점
    likes = [] # 좋아요 수
    comment = [] # 감상평
    page = 1 # 현재 크롤링할 페이지

    # 매개변수로 지정한 크롤링 페이지가 전체 페이지수보다 많을때 보정
    comment_num = driver.find_element(By.CSS_SELECTOR, "div.score_total > strong > em").text.replace(",","")
    max_page = int(comment_num) // 10
    if pagenum > max_page :
        pagenum = max_page

    # 본격적인 스크래핑 시작
    while page < pagenum :
        reviews = driver.find_elements(By.CSS_SELECTOR, "body > div > div > div.score_result > ul > li")
        idx = 0  # 현재 페이지에서 크롤링할 감상평 인덱스 값

        for review in reviews :
            # 평점
            score.append(review.find_element(By.CSS_SELECTOR, "div.star_score > em").text)
            # 좋아요 수
            likes.append(review.find_element(By.CSS_SELECTOR, "div.btn_area > a._sympathyButton > strong").text)
            # 감상평
            comment_text = review.find_element(By.CSS_SELECTOR, f"#_filtered_ment_{idx}").text
            # 너무 길어서 하이퍼링크 처리된 감상평 전문 가져오기
            if len(comment_text) >= 122 : 
                review.find_element(By.CSS_SELECTOR, f"#_filtered_ment_{idx} a").click()
                comment_text_full = review.find_element(By.CSS_SELECTOR, f"#_filtered_ment_{idx}").text
                comment.append(comment_text_full)
            # 그럴 필요 없는 것들 가져오기
            else : 
                comment.append(comment_text)
            idx += 1

        page += 1 
        driver.find_element(By.CSS_SELECTOR, "div.paging>div>a.pg_next").click()
        time.sleep(2) 

    #  셀레니움 종료
    driver.close()

    #  스크래핑한 결과를 데이터프레임 변수 result에 저장
    result["score"] = score
    result["likes"] = likes
    result["comment"] = comment
    print("스크래핑 결과\n", result.info())

    #  result를 csv파일로 저장
    result.to_csv(f"네이버영화크롤링_{filename}.csv", sep=",", index=False, encoding="utf-8")
    print("저장 완료!\n")

    # result를 리턴
    return result

In [ ]:
if __name__ == "__main__" :

    movienms = [
        "범죄도시2", "공조2", "한산",
        "늑대사냥", "외계인1부", "뜨거운피"]

    for movienm in movienms :
        get_reviews(movienm, filename = f"{movienm}", pagenum=200)